In [ ]:
# Import Libraries
import os
import web3
import asyncio

# Import funnctions from logic python files 
from logic.fetch_blocks import fetch_and_save_chunks
from logic.decode_userops import decode_all_userops_from_dir
from logic.decode_userops import append_native_value_to_userops
from logic.cluster_errors import (
    extract_all_error_transactions,
    cluster_from_block_errors,
    cluster_from_userops
)
from logic.visualize import visualize_userop_failures
from logic.faucet_click_analysis import generate_faucet_summary
from logic.faucet_visualize import plot_faucet_metrics
from logic.wallet_behavior import generate_wallet_behavior_summary



In [ ]:
# State variables
start_block = 1280000
end_block=1300000
rpc_url = "https://archive.testnet.incentiv.net"
chunck_size = 10000
batch_size=100
start_time="2025-06-12 23:13:09"
end_time="2025-06-30 07:53:09"
faucet_address="0x2feb013e33513d1f379b726011c950cae50fdd0a"
method_id="b03630d7"

In [ ]:
# Create the blocks folder which will store the block level csv files

os.makedirs("blocks", exist_ok=True)

await fetch_and_save_chunks(
    rpc_url=rpc_url,
    chunk_size=chunck_size,
    batch_size=batch_size,
    initial_chunk_number=1,
    output_dir="./blocks",
    start_block=start_block,
    end_block=end_block
)


In [ ]:
# Create the user ops csv files, data for each user operation individually which demonstrates every action from a wallet account

await decode_all_userops_from_dir(
    input_dir="blocks",
    rpc_url=rpc_url,
    output_dir="uops",
    batch_siz=batch_size
)

In [ ]:
# Adds an extrac column of native value, if it has been used, inside the user ops csv files

append_native_value_to_userops(
    folder="uops",
    rpc_url=rpc_url
)


In [ ]:
# Create the error summary file for block level

# Step 1: Extract error transactions
extract_all_error_transactions()

# Step 2: Only cluster if the error file exists
if os.path.exists("data/all_error_transactions.csv"):
    cluster_from_block_errors()
else:
    print("⚠️ No error transactions to cluster from block errors.")

# Step 3: Cluster from user operations (this is safe)
cluster_from_userops()

In [ ]:
# Visualizations regarding the failed user ops

visualize_userop_failures(
    sender_csv="data/summary_failures_per_sender.csv",
    bundler_csv="data/summary_failures_per_bundler.csv",
    paymaster_csv="data/summary_failures_per_paymaster.csv",
    output_prefix="final",
    start_block=start_block,
    end_block = end_block,
    start_time=start_time,
    end_time= end_time,
    output_dir="visualizations"
)


In [ ]:
# Generates the faucet summary (clicks, average time etc.)

generate_faucet_summary(
    rpc_url=rpc_url,
    faucet_address=faucet_address,
    method_id=method_id,
    start_block=start_block,
    end_block=end_block
)

In [ ]:
# Plots for faucet analysis 
plot_faucet_metrics()


In [ ]:
# Generates a csv file, which is analyzing each sender account and helps identify which of them are bots

generate_wallet_behavior_summary()
